In [110]:
from analysis import sns, COLORS, BEAM_COLORS, COLORS_C
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from mongoengine import connect
from pybirales.repository.models import SpaceDebrisTrack, Observation
from scipy.stats.stats import pearsonr
from numpy.linalg import lstsq
from pybirales.services.post_processing.processor import PostProcessor
from pybirales.services.post_processing.writer import TDMWriter
from datetime import datetime
from jinja2 import Environment, FileSystemLoader
import os


In [111]:
db_connection = connect(
    db='birales',
    username='birales_rw',
    password='arcadia10',
    port=27017,
    host='localhost')

In [112]:
TDM_OUT_DIR = '/home/denis/Desktop/'
SAVE_TDM = False

OBSERVATION = '5b8fac93be045a0d01b10c1e'
TARGET = '4071'

# OBSERVATION = '5b8fcd07be045a27a5414d8d'
# TARGET = '20439'

# OBSERVATION = '5b8fdcd4be045a33dbd511c0'
# TARGET = '37820'

REMOVE_DUPLICATE_EPOCH = True
REMOVE_DUPLICATE_CHANNEL = True

In [113]:
observation = Observation(id=OBSERVATION)
print 'Observation: {} {}'.format(observation.id, observation.name)
detected_candidates = SpaceDebrisTrack.get(observation_id=observation.id)

tracks = []
for candidate in detected_candidates:
    candidate.data = pd.DataFrame(data=candidate.data,
                                  columns=['time_sample', 'channel_sample', 'time', 'channel', 'snr',
                                           'beam_id'])
    print '\nCandidate: {}'.format(candidate.id)
    orig_size = len(candidate.data)
    if REMOVE_DUPLICATE_EPOCH:
        candidate.data = candidate.data.sort_values('snr', ascending=False).drop_duplicates(subset=['time_sample', 'beam_id']).sort_values(by=['time_sample'])
        
    print 'After epoch duplicate filter (size: {}/{})'.format(len(candidate.data), orig_size)

    if REMOVE_DUPLICATE_CHANNEL:
        candidate.data = candidate.data.sort_values('snr', ascending=False).drop_duplicates(subset=['channel_sample', 'beam_id']).sort_values(by=['time_sample'])
        
    print 'After channel duplicate filter (size: {}/{})'.format(len(candidate.data), orig_size)
    
    # Invalidate candidates based on the number of elements and number of illuminated beams
    if len(candidate.data) < 5 or len(candidate.data['beam_id'].unique()) < 2:
        print('Invalid: {} activated beams and size is {}'.format(len(candidate.data['beam_id'].unique()), len(candidate.data)))
        continue
        
    print('Valid: {} activated beams and size is {}'.format(len(candidate.data['beam_id'].unique()), len(candidate.data)))
    tracks.append(candidate)    

print '\nValid Candidates: {} (from {})'.format(len(tracks), len(detected_candidates))

Observation: 5b8fac93be045a0d01b10c1e None

Candidate: 5b8fac98be045a0d01b10c20
After epoch duplicate filter (size: 13/17)
After channel duplicate filter (size: 11/17)
Valid: 5 activated beams and size is 11

Candidate: 5b8facb0be045a0d01b10c31
After epoch duplicate filter (size: 3360/11193)
After channel duplicate filter (size: 3296/11193)
Valid: 32 activated beams and size is 3296

Valid Candidates: 2 (from 2)


In [114]:
if SAVE_TDM:
    template_dir = os.path.join('/home/denis/Development/birales/pybirales/services/post_processing')

    _template = Environment(loader=FileSystemLoader(template_dir)).get_template('input_template.tdm')

    for detection_num, track in enumerate(tracks):
        filepath = os.path.join(TDM_OUT_DIR, 'BIRALES_OUT_{:%Y%m%d}_{:0>3}.tdm'.format(min(track.data['time']), detection_num))
        data = dict(
            filename=TARGET,
            creation_date=datetime.utcnow().isoformat('T'),
            beams=np.unique(track.data['beam_id']),
            detection=track.data,
            target_name=TARGET,
            tx=track.tx,
            pointings=track.pointings,
            integration_interval=track.sampling_time
        )


        parsed_template = _template.render(**data)
        with open(filepath, "wb") as fh:
            fh.write(parsed_template)

            print('Output TDM {} persisted at: {}'.format(detection_num + 1, filepath))
